In [ ]:
# imports
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

In [ ]:
df_berlin = pd.read_csv('berlin_nextbike_final.csv')
df_berlin

In [ ]:
df_berlin[['latitude','longitude']] = df_berlin[['latitude','longitude']].apply(lambda x:round(x,4))
df_berlin.drop(columns='id')

In [ ]:
location_df = pd.DataFrame()
location_df['name'] = df_berlin['name']
location_df['latitude'] = df_berlin['latitude']
location_df['longitude'] = df_berlin['longitude']
location_df['latlon'] = location_df['latitude'].astype(str) + "," + location_df['longitude'].astype(str)
location_df

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
for idx, n in enumerate(location_df['name']):
    latlon = location_df['latlon'][idx]
    foursquare_url = f"https://places-api.foursquare.com/places/search?ll={latlon}&radius=1000"
    foursquare_key = os.environ['foursquare']

    headers = {
    "accept": "application/json",
    "X-Places-Api-Version": "2025-06-17",
    "authorization": f"Bearer {foursquare_key}"
    }

    response = requests.get(foursquare_url, headers=headers)
    foursquare_data = response.json()
    #foursquare_df = pd.json_normalize(foursquare_data['results'])
    with open(f'/foursquare_data/fs_data{idx}.json', 'w') as f:
        json.dump(foursquare_data, f, ensure_ascii=False)


In [ ]:
import glob
file_paths = glob.glob('/foursquare_data/*.json')

with open('output-file.json', 'w') as output_file:
    for file_path in file_paths:
        with open(file_path, 'r') as input_file:
            output_file.write(input_file.read() + '\n')

In [ ]:
with open('output-file.json', 'r') as f:
    data = [json.loads(line) for line in f]

In [ ]:
results = pd.json_normalize(data, record_path='results')
results.to_csv('results.csv')

In [ ]:
results.columns

In [ ]:
results['categories'] = results['categories'].apply(lambda lst: [d['name'] for d in lst])

In [ ]:
results_exploded = results.explode('categories')
results_exploded = results_exploded.rename(columns={'categories': 'category_name'})

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
results_exploded = results_exploded[results_exploded['category_name'].notnull()]

In [ ]:
results_exploded = results_exploded[results_exploded['category_name'].str.contains("Restaurant|Bar|Cafe|Caf|Museum|Gallery|Monastery|Church|Retail|Brewery|Winery|Bakery|Pub|Wine|Beer|Coffee")]

In [ ]:
results_exploded

Put your parsed results into a DataFrame

In [ ]:
berlin_df = pd.DataFrame()
berlin_df['fsq_place_id'] = results_exploded['fsq_place_id']
berlin_df['name'] = results_exploded['name']
berlin_df['category_name'] = results_exploded['category_name']
berlin_df['address'] = results_exploded['location.address']
berlin_df['postcode'] = results_exploded['location.postcode']
berlin_df['latitude'] = results_exploded['latitude']
berlin_df['longitude'] = results_exploded['longitude']
berlin_df['distance'] = results_exploded['distance']
berlin_df.to_csv('fsq_berlin_df.csv')

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
## I was unable to access the Yelp API so I didn't complete this part of the exercise.

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating